In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 13:06:04.658302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 13:06:04.764926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 13:06:04.764944: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 13:06:04.786309: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 13:06:05.318854: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 13:06:09.695747: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 13:06:09.696300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:06:09.696350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:06:09.696389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 13:06:09.696428: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 395ms/step - loss: 3.5479 - auc: 0.2173
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 2.0285 - auc: 0.4192
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4047 - auc: 0.7805
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3900 - auc: 0.8586
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 1.5187 - auc: 0.8766
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6855 - auc: 0.8693
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 1.7079 - auc: 0.8751
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.7047 - auc: 0.8756
Epoch 9/100
1/1 [==============================] - 0s 28ms/step - loss: 1.6924 - auc: 0.8763
Epoch 10/100
1/1 [==============================] - 0s 37ms/step - loss: 1.5713 - auc: 0.8909
Epoch 11/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4624 

1/1 [==============================] - 0s 30ms/step - loss: 0.2640 - auc: 0.9905
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2639 - auc: 0.9895
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2637 - auc: 0.9902
Epoch 91/100
1/1 [==============================] - 0s 33ms/step - loss: 0.2687 - auc: 0.9884
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2666 - auc: 0.9887
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2663 - auc: 0.9893
Epoch 94/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2576 - auc: 0.9894
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2598 - auc: 0.9893
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2598 - auc: 0.9897
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2531 - auc: 0.9905
Epoch 98/100
1/1 [==============================] - 0s 21ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
      'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 341ms/step - loss: 1.5263 - auc: 0.6232
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4073 - auc: 0.7974
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4181 - auc: 0.8193
Epoch 4/100
1/1 [==============================] - 0s 32ms/step - loss: 1.3163 - auc: 0.8337
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1915 - auc: 0.8413
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0987 - auc: 0.8395
Epoch 7/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0789 - auc: 0.8215
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 1.0371 - auc: 0.8261
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9890 - auc: 0.8432
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.8857 - auc: 0.8934
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2748 - auc: 0.9862
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2812 - auc: 0.9851
Epoch 90/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2736 - auc: 0.9869
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2721 - auc: 0.9868
Epoch 92/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2783 - auc: 0.9854
Epoch 93/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2736 - auc: 0.9866
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2644 - auc: 0.9886
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2622 - auc: 0.9890
Epoch 96/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2740 - auc: 0.9866
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2704 - auc: 0.9871
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 337ms/step - loss: 1.5117 - auc: 0.6577
Epoch 2/100
1/1 [==============================] - 0s 28ms/step - loss: 1.4285 - auc: 0.8144
Epoch 3/100
1/1 [==============================] - 0s 35ms/step - loss: 1.3728 - auc: 0.8293
Epoch 4/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2257 - auc: 0.8493
Epoch 5/100
1/1 [==============================] - 0s 37ms/step - loss: 1.1219 - auc: 0.8517
Epoch 6/100
1/1 [==============================] - 0s 38ms/step - loss: 1.0427 - auc: 0.8584
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 0.9903 - auc: 0.8718
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9475 - auc: 0.8856
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8608 - auc: 0.9209
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 0.7915 - auc: 0.9445
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2344 - auc: 0.9915
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2322 - auc: 0.9917
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2352 - auc: 0.9906
Epoch 91/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2354 - auc: 0.9908
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2364 - auc: 0.9903
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2363 - auc: 0.9908
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2423 - auc: 0.9894
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2350 - auc: 0.9907
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2398 - auc: 0.9903
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2313 - auc: 0.9913
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [6]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 2.1516 - auc: 0.3341
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.3973 - auc: 0.7112
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3548 - auc: 0.8198
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4507 - auc: 0.8388
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4850 - auc: 0.8499
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4752 - auc: 0.8506
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.3685 - auc: 0.8607
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2725 - auc: 0.8657
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1261 - auc: 0.8744
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 1.0224 - auc: 0.8772
Epoch 11/100
1/1 [==============================] - 0s 26ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2922 - auc: 0.9830
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2935 - auc: 0.9825
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2913 - auc: 0.9837
Epoch 91/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2919 - auc: 0.9836
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2959 - auc: 0.9819
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2910 - auc: 0.9833
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2980 - auc: 0.9818
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2880 - auc: 0.9838
Epoch 96/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2841 - auc: 0.9848
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2930 - auc: 0.9820
Epoch 98/100
1/1 [==============================] - 0s 30ms/step - loss: 

In [7]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 1.4485 - auc: 0.7364
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3272 - auc: 0.7767
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.2246 - auc: 0.7876
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.1119 - auc: 0.8539
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0530 - auc: 0.8647
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 0.9766 - auc: 0.8802
Epoch 7/100
1/1 [==============================] - 0s 35ms/step - loss: 0.9265 - auc: 0.8937
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 0.8371 - auc: 0.9258
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8027 - auc: 0.9349
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 0.7628 - auc: 0.9435
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2465 - auc: 0.9898
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2517 - auc: 0.9889
Epoch 90/100
1/1 [==============================] - 0s 40ms/step - loss: 0.2364 - auc: 0.9921
Epoch 91/100
1/1 [==============================] - 0s 38ms/step - loss: 0.2386 - auc: 0.9917
Epoch 92/100
1/1 [==============================] - 0s 32ms/step - loss: 0.2389 - auc: 0.9916
Epoch 93/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2382 - auc: 0.9913
Epoch 94/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2442 - auc: 0.9902
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2489 - auc: 0.9898
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2368 - auc: 0.9921
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2349 - auc: 0.9915
Epoch 98/100
1/1 [==============================] - 0s 42ms/step - loss: 

In [8]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 344ms/step - loss: 1.5530 - auc: 0.5913
Epoch 2/100
1/1 [==============================] - 0s 29ms/step - loss: 1.3734 - auc: 0.7984
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4081 - auc: 0.8214
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.2960 - auc: 0.8423
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1926 - auc: 0.8464
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0787 - auc: 0.8562
Epoch 7/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0543 - auc: 0.8388
Epoch 8/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9794 - auc: 0.8567
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9646 - auc: 0.8530
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 0.9074 - auc: 0.8771
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2678 - auc: 0.9877
Epoch 89/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2632 - auc: 0.9892
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2665 - auc: 0.9886
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2660 - auc: 0.9883
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2690 - auc: 0.9875
Epoch 93/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2622 - auc: 0.9894
Epoch 94/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2584 - auc: 0.9895
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2680 - auc: 0.9880
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2627 - auc: 0.9882
Epoch 97/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2583 - auc: 0.9902
Epoch 98/100
1/1 [==============================] - 0s 26ms/step - loss: 

In [9]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.4710 - auc: 0.7770
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3461 - auc: 0.7502
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 1.2137 - auc: 0.8095
Epoch 4/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1413 - auc: 0.8510
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.0328 - auc: 0.8895
Epoch 6/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9682 - auc: 0.9015
Epoch 7/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9123 - auc: 0.9154
Epoch 8/100
1/1 [==============================] - 0s 31ms/step - loss: 0.8646 - auc: 0.9271
Epoch 9/100
1/1 [==============================] - 0s 26ms/step - loss: 0.7952 - auc: 0.9483
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 0.7467 - auc: 0.9598
Epoch 11/100
1/1 [==============================] - 0s 37ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.2269 - auc: 0.9913
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2209 - auc: 0.9934
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2289 - auc: 0.9915
Epoch 91/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2332 - auc: 0.9900
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2235 - auc: 0.9928
Epoch 93/100
1/1 [==============================] - 0s 34ms/step - loss: 0.2262 - auc: 0.9914
Epoch 94/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2267 - auc: 0.9909
Epoch 95/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2263 - auc: 0.9920
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2284 - auc: 0.9908
Epoch 97/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2233 - auc: 0.9920
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [10]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('6feature_remove_one3.csv',index=False)
print(de)

               model       auc
0          all_model  0.947924
1  remove_AtomPairFP  0.921890
2      remove_Charge  0.946694
3        remove_MAP4  0.944848
4         remove_MOE  0.947683
5    remove_MorganFP  0.948011
6    remove_Topology  0.947732
